In [45]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Imports

In [46]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), os.pardir))
from Tools.leica_tools import parse_lif
from Tools.exp_tools import Experiment

In [15]:
#parse_lif('/Volumes/T7/DMI/NKIP_FA_083/NKIP_FA_083.lif')

In [18]:
expID = 'NKIP_FA_084'
experiment = Experiment(expID)
experiment.detect_droplets_legacy(mode='sweep')

In [19]:
experiment.cell_count(model_name='cell_count_v3.h5')

Cause: generators are not supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: generators are not supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2025-08-07 14:47:55.120410: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


3835/3835 ━━━━━━━━━━━━━━━━━━━━ 401s 105ms/step


2025-08-07 14:54:36.204914: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/Users/fauberma/mambaforge/envs/phenotyping_v1/lib/python3.12/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


In [41]:
experiment.generate_ap(labels=['target', 'effector', 'dead_target','dead_effector'], size=400, subset_query='time == 8')

In [ ]:
#experiment.remove_ap('AP_0')

In [62]:
import pandas as pd
import numpy as np
existing_annotations = experiment.db.get_annotations(source='manual')
ap_id = 'AP_2'
droplet_ids=None
subset_query='index == index'
size=400
labels=['target', 'effector', 'dead_target','dead_effector']
if ap_id in existing_annotations['ap_id'].unique():
    raise Exception(f'Annotation package with id "{ap_id}" exists already.')

if droplet_ids is None:
    droplet_df = experiment.get_droplet_df().drop(index=existing_annotations['droplet_id'].unique())
    droplet_ids = droplet_df.query(subset_query).sample(size).index

annotations = pd.DataFrame(index=pd.MultiIndex.from_product([droplet_ids, labels], names=['droplet_id', 'label_type']), 
                           columns=['value', 'ap_id', 'status'],
                           data=[[np.nan, ap_id, 'pending']])




In [69]:
experiment.get_droplet_df().query(f'index not in []')

,experiment_id,frame_id,x_min,y_min,x_max,y_max,outlier,time,condition
droplet_id,,,,,,,,,
0,NKIP_FA_084,0,5106,14745,5244,14883,False,1,Bulk
1,NKIP_FA_084,0,3751,3632,3889,3770,False,1,Bulk
2,NKIP_FA_084,0,5626,3548,5764,3686,False,1,Bulk
3,NKIP_FA_084,0,2857,3465,2995,3603,False,1,Bulk
4,NKIP_FA_084,0,5498,3455,5636,3593,False,1,Bulk
...,...,...,...,...,...,...,...,...,...
122693,NKIP_FA_084,31,5669,16058,5791,16180,False,8,Bulkstain
122694,NKIP_FA_084,31,3549,2905,3681,3037,False,8,Bulkstain
122695,NKIP_FA_084,31,97,464,227,594,False,8,Bulkstain


In [68]:
experiment.get_droplet_df()

,experiment_id,frame_id,x_min,y_min,x_max,y_max,outlier,time,condition
droplet_id,,,,,,,,,
0,NKIP_FA_084,0,5106,14745,5244,14883,False,1,Bulk
1,NKIP_FA_084,0,3751,3632,3889,3770,False,1,Bulk
2,NKIP_FA_084,0,5626,3548,5764,3686,False,1,Bulk
3,NKIP_FA_084,0,2857,3465,2995,3603,False,1,Bulk
4,NKIP_FA_084,0,5498,3455,5636,3593,False,1,Bulk
...,...,...,...,...,...,...,...,...,...
122693,NKIP_FA_084,31,5669,16058,5791,16180,False,8,Bulkstain
122694,NKIP_FA_084,31,3549,2905,3681,3037,False,8,Bulkstain
122695,NKIP_FA_084,31,97,464,227,594,False,8,Bulkstain
